<h2>Example of DashLeaflet displaying Geotiff with TiTiler<h2>

In [32]:
#Import PLotly Dash , Dash Leaflet (for mapping), Jupyter Dash (for using dash/dash leaflet in notebook),
# IMport  Json & httpx for getting data from TiTiler 
from jupyter_dash import JupyterDash
from dash import Dash, html,dcc,Input, Output
import dash_leaflet as dl
import json
import httpx

In [33]:
#Set TiTiler endpoint as local host 
#Titiler is running using Docker Image 
#Local host is running on port 8000 , this is set in Docker desktop when running the image. 
#Url refers to a s3 bucket 
#Note: .tiff is a Cloud Optomised Geotiff[COG] , .tiff must be a COG!!!

titiler_endpoint = "http://localhost:8000"  # titiler docker image running on local .
url = "https://rapidepe123.s3.eu-west-2.amazonaws.com/GBR_wind-speed_100m_COGv2.tif"


In [34]:
# Extract min and max values of the COG , this is used to rescale the COG
r = httpx.get(
    f"{titiler_endpoint}/cog/statistics",
    params = {
        "url": url,
    }
).json()

minv = (r["1"]["min"])
maxv = (r["1"]["max"])


minv,maxv

(4.905625820159912, 16.434234619140625)

In [35]:
#get tile map from titiler endpoint and url , and scale to min and max values 

r = httpx.get(
    f"{titiler_endpoint}/cog/tilejson.json",
    params = {
        "url": url,
        "rescale": f"{minv},{maxv}",
        "colormap_name": "viridis"
    }
).json()



In [36]:
#create function which uses "get point" via titiler to get point value . This will be used in dash to get 
#point value and present this on the map 
# note "nearest" lat lon will be used by

def get_point_value(lat,lon):

    Point = httpx.get(
        f"{titiler_endpoint}/cog/point/{lon},{lat}",
        params = {
            "url": url,
            "resampling": "average"

        }
    ).json()


    return "{:.2f}".format(float(Point["values"][0]))

#test function is working 
x = get_point_value(55.5,-4)
x

'10.10'

In [38]:
#Build Dash app (note: JupyterDash being used here but will work in external dash app , to see this change last line to "mode=External")

app = JupyterDash(__name__)

#create the info panel where our wind speed will be displayed 
info = html.Div( id="info", className="info",
                style={"position": "absolute", "bottom": "10px", "left": "10px", "z-index": "1000"})

#Create app layout 
#dash leaflet used to create map (https://dash-leaflet.herokuapp.com/)
#dl.LayersControl, dl.overlay & dl.LayerGroup used to add layer selection functionality 
app.layout = html.Div([
dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[55, -4],
               zoom=5,
               id = "map",
               children=[
            dl.LayersControl([
                
                   dl.Overlay(dl.LayerGroup(dl.TileLayer(url="https://cartodb-basemaps-{s}.global.ssl.fastly.net/light_nolabels/{z}/{x}/{y}.png",id="TileMap")),name="BaseMap",checked=True),
                   #COG fed into Tilelayer using TiTiler url (taken from r["tiles"][0])
                   dl.Overlay(dl.LayerGroup(dl.TileLayer(url=r["tiles"][0], opacity=0.8,id="WindSpeed@100m")),name="WS@100m",checked=True),
                   dl.LayerGroup(id="layer"),
                # set colorbar and location in app            
                   dl.Colorbar(colorscale="viridis", width=20, height=150, min=minv, max=maxv,unit='m/s',position="bottomright"),
                   info,
                   
])
])
])
# create callback which uses inbuilt "click_lat_lng" feature of Dash leaflet map, extract lat/lon and feed to get_point_value function , WS fed to output id = info (which is info html.div)
@app.callback(Output("info", "children"), [Input("map", "click_lat_lng")])
def map_click(click_lat_lng):

    lat= click_lat_lng[0]
    lon=click_lat_lng[1]
    
    return get_point_value(lat,lon)



app.run_server(debug=True,mode='inline')
